In [7]:
import tqdm

import numpy as np

import src

In [41]:
train_models = src.shapenet.get_shapenet_data("train")
for i in tqdm.tqdm(range(10)):
    model_path = train_models[np.random.randint(len(train_models))]
    cloud = src.shapenet.sample_points(model_path, nb=10000)

100%|██████████| 10/10 [00:02<00:00,  3.81it/s]


In [42]:
from sklearn.neighbors import KDTree

In [82]:
model_path = train_models[np.random.randint(len(train_models))]    

In [98]:
def angle(u,v):
    cross = np.linalg.norm(np.cross(u, v), axis=-1)
    dot = np.sum(u*v, axis=-1)
    return np.arctan2(cross, dot)


class PPFPatchExtractor:
    """
    From a point cloud:
    Sample the points that will be centers
    Using KD-tree find neighborhoods
    Then compute PPF on them.
    """
    def __init__(self, nb_patches=512, nb_points=256, tau=0.1):
        self.nb_patches = nb_patches
        self.nb_points = nb_points
        self.tau = tau
        
    def make_patches(self, cloud):
        points, normals = cloud
        centers_idx = np.random.choice(len(points), size=self.nb_patches, replace=False)
        centers = points[centers_idx]
        kd_tree = KDTree(points) 
        indexes = kd_tree.query(centers, k=self.nb_points, return_distance=False)
        patches = points[indexes]
        centers_normals = normals[centers_idx]
        patches_normals = normals[indexes]
        return centers, centers_normals, patches, patches_normals

    def compute_ppf(self, centers, centers_normals, patches, patches_normals):
        centers = centers[:,np.newaxis,:]
        centers_normals = centers_normals[:,np.newaxis,:]
        delta = patches - centers
        dist = np.linalg.norm(delta, axis=-1)
        angle_0 = angle(centers_normals, delta) 
        angle_1 = angle(patches_normals, delta) 
        angle_2 = angle(centers_normals, patches_normals) 
        ppf = np.stack((dist, angle_0, angle_1, angle_2), axis=-1)
        features = np.concatenate((delta, patches_normals, ppf), axis=-1)
        return features
    
    def compute_M(self, centers):
        diff = centers[:,np.newaxis] - centers[np.newaxis,:]
        dist = np.sum(diff*diff, axis=-1)
        M = dist < self.tau*self.tau
        return M
    
    def make_example(self, cloud):
        """
        Transform point cloud with normals into PPF.
        """
        centers, centers_normals, patches, patches_normals =  self.make_patches(cloud)
        ppf = self.compute_ppf(centers, centers_normals, patches, patches_normals)
        M = self.compute_M(centers)
        return ppf, M
    
    def get_N_tuple_loss(self):
        @tf.function
        def loss(M, pred):
            pass

extractor = PPFPatchExtractor()
res = extractor.make_example(cloud)

In [13]:
nb
patch_centers = np.random.choice(len(points), size=)

(array([[-0.05049742,  0.001337  ,  0.17563553],
        [-0.05518185,  0.01112944, -0.08736867],
        [ 0.20113309,  0.007514  , -0.0510883 ],
        ...,
        [-0.07569871,  0.01077688, -0.0801021 ],
        [-0.0385164 ,  0.00343553,  0.167939  ],
        [-0.01597736,  0.02162101, -0.17428771]]),
 array([[-0.        , -1.        ,  0.        ],
        [-0.01718161,  0.99985239,  0.        ],
        [ 0.        , -1.        ,  0.        ],
        ...,
        [ 0.01718161, -0.99985239,  0.        ],
        [ 0.        ,  0.        ,  1.        ],
        [-0.78951488,  0.61361096,  0.01215885]]))